In [2]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/samples.ipynb")

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


In [13]:
len(normal_samples), len(normal_barcodes)

(8333, 8333)

In [14]:
sample_dictionary = {}
for i, barcode in enumerate(normal_barcodes):
    sample_dictionary[barcode] = normal_samples[i]

In [15]:
allele_dictionary = pickle.load(open('/cellar/users/ramarty/Data/hla_ii/hla_types/TCGA.HLA_classII.p', "rb"))
len(allele_dictionary.keys())

8262

In [16]:
untyped_barcodes = [x for x in normal_barcodes if x not in allele_dictionary.keys()]
len(untyped_barcodes)

71

In [17]:
untyped_samples = [sample_dictionary[x] for x in untyped_barcodes]

In [18]:
out_dirs = ['/nrnb/users/ramarty/TCGA/exomes/{0}'.format(x) for x in untyped_barcodes]

In [19]:
print len(out_dirs), len(untyped_samples), len(untyped_barcodes)

71 71 71


In [10]:
out_dirs[:10]

['/nrnb/users/ramarty/TCGA/exomes/TCGA-DD-A1EH',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-13-1511',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-DV-5576',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-13-0891',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-AG-3602',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-AA-3877',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-AG-3574',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-60-2721',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-60-2720',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-10-0926']

In [11]:
def create_cluster_script_download(samples, barcodes, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/mhc_types/exon_download.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        #out_file.write("#$ -t 1-2\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=8G\n")
        out_file.write("#$ -tc 50\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make directory
        out_file.write("mkdir $out\n")
        out_file.write("mkdir $out/hlaHD\n")
        out_file.write("mkdir /tmp/$barcode\n")
        out_file.write("\n")
        
        # Download and process
        out_file.write("bash /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/GDC.exome.sh $sample /tmp/$barcode/full_exome.bam\n")
        out_file.write("echo Bam downloaded.\n")
        out_file.write("\n")
        
        # Index bam
        out_file.write("/cellar/users/ramarty/programs/samtools-1.3.1/samtools index /tmp/$barcode/full_exome.bam\n")
        out_file.write("echo index.\n")
        out_file.write("\n")
        
        # Slice HLA
        out_file.write('/cellar/users/ramarty/programs/samtools-1.3.1/samtools view -b /tmp/$barcode/full_exome.bam "chr6:22361036-47685581" > $out/chr6.bam\n')
        out_file.write("\n")
        
        # Slice KIR
        out_file.write('/cellar/users/ramarty/programs/samtools-1.3.1/samtools view -b /tmp/$barcode/full_exome.bam "chr19:54051467-55502666" > $out/chr19.bam\n')
        out_file.write("\n")
        
        # Slice unmapped
        out_file.write("/cellar/users/ramarty/programs/samtools-1.3.1/samtools view -b -f 4 /tmp/$barcode/full_exome.bam > $out/unmapped.bam\n")
        out_file.write("echo sliced.\n")
        out_file.write("\n")
        
        # Merge bam
        out_file.write('/cellar/users/ramarty/programs/samtools-1.3.1/samtools merge /tmp/$barcode/merged.bam $out/chr6.bam $out/unmapped.bam\n')
        out_file.write("echo merged.\n")
        out_file.write("\n")
        
        # Strip bam
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq2.py /tmp/$barcode/merged.bam /tmp/$barcode/merged_sorted /tmp/$barcode/merged_1.fastq /tmp/$barcode/merged_2.fastq cellar\n")
        out_file.write("echo Fastq stripped.\n")
        out_file.write("\n")
        
        # HLA-HD
        out_file.write("hlahd.sh -t 8 -m 70 -f ~/programs/hlahd.1.0.0/freq_data/ /tmp/$barcode/merged_1.fastq /tmp/$barcode/merged_2.fastq ~/programs/hlahd.1.0.0/HLA_gene.split.txt ~/programs/hlahd.1.0.0/dictionary/ sampleID $out/hlaHD\n")
        out_file.write("echo HLA-HD completed.\n")
        out_file.write("\n")
        
        out_file.write("rm $out/merged*\n")
        out_file.write("rm $out/full_exome*\n")
        out_file.write("rm -r /tmp/$barcode/*\n")
        out_file.write("date\n")

In [12]:
#create_cluster_script_download(normal_samples, normal_barcodes, out_dirs)
create_cluster_script_download(untyped_samples[2:], untyped_barcodes[2:], out_dirs[2:])

In [13]:
def create_cluster_script_gather(samples, barcodes, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/mhc_types/run_hlaHD.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 2354-{0}\n".format(len(samples)))
        out_file.write("#$ -t 2354-2355\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=30G\n")
        out_file.write("#$ -tc 8\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        

In [14]:
create_cluster_script_gather(normal_samples, normal_barcodes, out_dirs)

In [14]:
def create_cluster_script_delete(samples, barcodes, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/mhc_types/delete_exomes.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 2000-{0}\n".format(len(samples)))
        #out_file.write("#$ -t 2354-2355\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=1G\n")
        out_file.write("#$ -tc 100\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Clean up
        out_file.write("rm $out/full_exome*.fastq\n")
        out_file.write("rm $out/full_exome_sorted*\n")
        out_file.write("rm $out/full_exome*bam\n")
        out_file.write("date\n")

In [15]:
create_cluster_script_delete(normal_samples, normal_barcodes, out_dirs)

Check success

In [2]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/hla_types/hla_types.tcga.csv', index_col=0)

In [3]:
len(df)

2279

In [12]:
df.head()

,DPA1_allele1,DPA1_allele2,DPB1_allele1,DPB1_allele2,DQA1_allele1,DQA1_allele2,DQB1_allele1,DQB1_allele2,DRB1_allele1,DRB1_allele2
TCGA-02-0055,DPA10103,DPA10103,DPB10402,DPB10401,DQA10505,DQA10301,DQB10301,DQB10301,DRB1_1101,DRB1_0407
TCGA-02-2466,DPA10103,DPA10103,DPB10201,DPB10401,DQA10501,DQA10101,DQB10501,DQB10201,DRB1_0301,DRB1_0101
TCGA-02-2483,DPA10103,DPA10103,DPB10201,DPB10401,DQA10101,DQA10301,DQB10501,DQB10302,DRB1_0403,DRB1_0101
TCGA-04-1336,DPA10201,DPA10201,DPB11301,DPB10101,DQA10201,DQA10102,DQB10604,DQB10202,DRB1_1302,DRB1_0701
TCGA-04-1347,DPA10201,DPA10103,DPB10401,DPB11701\n,DQA10401,DQA10201\n,DQB10202,DQB10402,DRB1_0701,DRB1_0801


In [13]:
patient_dictionary = pickle.load(open('/cellar/users/ramarty/Data/hla_ii/hla_types/TCGA.HLA_classII.p'))

In [14]:
patient_dictionary

{'TCGA-D3-A3CE': ['DRB1_0101',
  'DRB1_1501',
  'HLA-DPA10103_DPB10401',
  'HLA-DPA10103_DPB10301',
  'HLA-DPA10103_DPB10401',
  'HLA-DPA10103_DPB10301',
  'HLA-DQA10102_DQB10602',
  'HLA-DQA10102_DQB10501',
  'HLA-DQA10101_DQB10602',
  'HLA-DQA10101_DQB10501'],
 'TCGA-D3-A3CB': ['DRB1_0101',
  'DRB1_0101',
  'HLA-DPA10201_DPB11701',
  'HLA-DPA10201_DPB10301',
  'HLA-DPA10103_DPB11701',
  'HLA-DPA10103_DPB10301',
  'HLA-DQA10101_DQB10501',
  'HLA-DQA10101_DQB10501',
  'HLA-DQA10101_DQB10501',
  'HLA-DQA10101_DQB10501'],
 'TCGA-DH-A7UV': ['DRB1_1101',
  'DRB1_1101',
  'HLA-DPA10103_DPB10401',
  'HLA-DPA10103_DPB110401',
  'HLA-DPA10103_DPB10401',
  'HLA-DPA10103_DPB110401',
  'HLA-DQA10505_DQB10301',
  'HLA-DQA10505_DQB10301',
  'HLA-DQA10601_DQB10301',
  'HLA-DQA10601_DQB10301'],
 'TCGA-C5-A1BM': ['DRB1_1201',
  'DRB1_0701',
  'HLA-DPA10103_DPB11801',
  'HLA-DPA10103_DPB10301',
  'HLA-DPA10103_DPB11801',
  'HLA-DPA10103_DPB10301',
  'HLA-DQA10201_DQB10501',
  'HLA-DQA10201_DQB10202',
 

In [5]:
l = [6.4, 6.4, 27.1, 27.1, 23.1, 80.4, 56.2, 45.2, 94.3, 5.3, 35.1, 73.2]
s = 0
for m in l:
    s += 1/m
len(l) / s

16.588768340705542

In [6]:
s

0.7233810101835219